<a href="https://colab.research.google.com/github/DonghaeSuh/machine-learning/blob/main/KNN%EC%82%AC%EC%A7%84%EB%B6%84%EB%A5%98%20%EB%B0%8F%20multivariate%20polynomial%20regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from skimage.io import imread 
from skimage.transform import resize

url = 'http://github.com/dknife/ML/raw/main/data/Proj1/40/'

imgR,imgC,channel = 24,24,3
images=[]

for i in range(40):
  file = url+'img{:02d}.jpg'.format(i+1)
  img = imread(file)
  img = resize(img,(imgR,imgC,channel))
  images.append(img)

In [ ]:
def plot_images(nRow,nCol,images):
  fig, ax= plt.subplots(nRow,nCol,figsize = (nCol,nRow))

  for i in range(nRow):
    for j in range(nCol):
      if nRow <=1: axes=ax[j]
      else:        axes=ax[i,j]
      axes.get_xaxis().set_visible(False)
      axes.get_yaxis().set_visible(False)
      axes.imshow(images[i*nCol+j])
plot_images(4,10,images)

노이즈 사진 만들기

In [ ]:
X = np.array(images[:30])
print(X.shape)
X_noisy = X+np.random.randn(len(X), imgR, imgC, channel)*0.2
y_label = np.array(X*255,dtype= np.uint)
y=y_label

for i in range(100):
  noisy_data = X + np.random.randn(len(X),imgR,imgC,channel)*0.2
  X_noisy = np.append(X_noisy,noisy_data,axis=0)
  y = np.append(y,y_label,axis=0)

X_noisy = np.clip(X_noisy,0,1)

plot_images(1,10,X_noisy[0:300:30])
X_noisy.shape,y.shape

In [ ]:
np.array(images[0]).shape

K-NN 적용

In [ ]:

X_noisy_flat = X_noisy.reshape(-1,imgR*imgC*channel)
y_flat = y.reshape(-1,imgR*imgC*channel)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_noisy_flat,y_flat)

In [ ]:
rndidx = np.random.randint(0,20)
data = X[rndidx:rndidx+10] + np.random.randn(10,imgR,imgC,channel)*0.4
data = np.clip(data,0,1)
data_flat = data.reshape(-1,imgR*imgC*channel)

denoised=knn.predict(data_flat)
denoised=denoised.reshape(-1,imgR,imgC,channel)

plot_images(1,10,data)
plot_images(1,10,denoised)

multivariate polynomial regression

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.linear_model import LinearRegression
data_loc = 'http://github.com/dknife/ML/raw/main/data/'
df = pd.read_csv(data_loc + 'nonlinear.csv')

plt.scatter(df['x'],df['y'])

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

X=df['x'].to_numpy()
y=df['y'].to_numpy()

X=X.reshape(-1,1)
feature_cubic=PolynomialFeatures(degree=3)
X_3 = feature_cubic.fit_transform(X)

lin_model = LinearRegression()
lin_model.fit(X_3,y)

In [ ]:
domain  = np.linspace(0,1,100).reshape(-1,1)
domain_3 = feature_cubic.fit_transform(domain)
predictions = lin_model.predict(domain_3)

plt.scatter(domain,predictions,color='r')